In [1]:
import pandas as pd
import numpy as np
from sklearn.externals.array_api_compat.torch import remainder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import category_encoders as ce

In [2]:
df = pd.read_csv('../data/gurgaon_properties_post_feature_selection_v2.csv')

In [3]:
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [5]:
X = df.drop(columns=['price'])
y = df['price']

In [7]:
y_transformed = np.log1p(y)

In [8]:
param_grid = {
    'regressor__n_estimators': [200, 400, 600],
    'regressor__learning_rate': [0.01, 0.05, 0.1],
    'regressor__max_depth': [3, 5, 7],
    'regressor__min_child_weight': [1, 3, 5],
    'regressor__subsample': [0.7, 0.9, 1.0],
    'regressor__colsample_bytree': [0.7, 1.0],
    'regressor__gamma': [0, 0.1, 0.3],
    'regressor__reg_alpha': [0, 0.001, 0.01],
    'regressor__reg_lambda': [1, 5, 10]
}


In [9]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ],
    remainder='passthrough'
)

In [11]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

In [17]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
search = GridSearchCV(
    pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4
)

search.fit(X, y_transformed)

Fitting 10 folds for each of 13122 candidates, totalling 131220 fits


,estimator,"Pipeline(step...=None, ...))])"
,param_grid,"{'regressor__colsample_bytree': [0.7, 1.0], 'regressor__gamma': [0, 0.1, ...], 'regressor__learning_rate': [0.01, 0.05, ...], 'regressor__max_depth': [3, 5, ...], ...}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,4
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...), ...]"


In [18]:
final_pipe = search.best_estimator_

In [19]:
search.best_params_

{'regressor__colsample_bytree': 0.7,
 'regressor__gamma': 0,
 'regressor__learning_rate': 0.05,
 'regressor__max_depth': 7,
 'regressor__min_child_weight': 1,
 'regressor__n_estimators': 400,
 'regressor__reg_alpha': 0.01,
 'regressor__reg_lambda': 1,
 'regressor__subsample': 0.7}

In [20]:
search.best_score_

np.float64(0.9110026206972763)

In [21]:
final_pipe.fit(X, y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
